# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2024-04-07 06:46:23 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


# Initial Solver
Run Installation First


In [ ]:
import Pkg; Pkg.add("CSV")
import Pkg; Pkg.add("Tables")
import Pkg; Pkg.add("JuMP")
import Pkg; Pkg.add("HiGHS")

import CSV
import Tables
import JuMP
import HiGHS
import JuMP.Containers
using CSV
using Tables
using JuMP
using HiGHS
using JuMP.Containers

In [ ]:
input_evacuee_matrix = CSV.File("input_evacuee_matrix.csv") |> Tables.matrix
input_shelter_matrix = CSV.File("input_shelter_matrix.csv") |> Tables.matrix

evacuee_names = input_evacuee_matrix[1:990,1]
shelter_names = input_shelter_matrix[:,1]

In [ ]:
full_cost_matrix = CSV.File("final_cost_matrix.csv") |> Tables.matrix

edited_cost_matrix = full_cost_matrix[:,2:end]

full_cost_array = Containers.DenseAxisArray(edited_cost_matrix, 1:990, 1:187)

shelter_numbers = input_shelter_matrix[1:end,2]

evacuee_numbers = input_evacuee_matrix[1:end,2]

function solve_transportation_problem(data::Containers.DenseAxisArray,shelter_numbers,evacuee_numbers)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    #O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == evacuee_numbers[o])
    @constraint(model, [d in D], sum(x[:, d]) <= shelter_numbers[d])
    optimize!(model)


    # Construct the solution matrix
    solution_matrix = zeros(length(O), length(D))  # Initialize with zeros

    for (i, o) in enumerate(O)
        for (j, d) in enumerate(D)
            solution_matrix[i, j] = value(x[o, d])
        end
    end

    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return solution_matrix
end

print("\nSolution:\n")
solution_matrix = solve_transportation_problem(full_cost_array, shelter_numbers, evacuee_numbers)
println(solution_matrix)

CSV.write("sendai_solution_matrix.csv", Tables.table(solution_matrix))


In [ ]:
#NOT DONE Total Cost Calculation
solution_cost = full_cost_array.data[1:end-1,1:end-1]
matching_indicies = findall(x-> x == ".", solution_cost)
solution_cost[matching_indicies] .= 0
#solution_matrix .* solution_cost
total_cost_matrix = solution_cost.*solution_matrix
running_total = sum(total_cost_matrix)
initial_running_total = running_total
print("Average distance per evacuee: ")
println(initial_running_total/sum(evacuee_numbers))

Average distance per evacuee: 0.9297263176521238


# Testing

In [13]:
function is_evacuee_numbers_matching(solution_matrix,evacuee_numbers)
  column_wise_solution_sum = sum(solution_matrix, dims=2)
  int_column_wise_solution_sum = Int.(column_wise_solution_sum)
  evacuees_comparison = int_column_wise_solution_sum .== evacuee_numbers
  return prod(evacuees_comparison)
end

is_evacuee_numbers_matching(solution_matrix,evacuee_numbers)

true

In [12]:
function is_shelter_not_overflowing(solution_matrix,shelter_numbers)
  row_wise_solution_sum = sum(solution_matrix, dims=1)
  int_row_wise_solution_sum = Int.(row_wise_solution_sum)
  shelter_comparison = int_row_wise_solution_sum[1,:] .<= shelter_numbers
  return prod(shelter_comparison)
end

is_shelter_not_overflowing(solution_matrix,shelter_numbers)

true

In [ ]:
#Evacuee = shelter_max +1 overload check
println(sum(shelter_numbers))
println(sum(evacuee_numbers))

shelter_evacuee_difference = sum(shelter_numbers) - sum(evacuee_numbers)
overload_evacuee_numbers = deepcopy(evacuee_numbers)
overload_evacuee_numbers[1] += shelter_evacuee_difference

println(sum(overload_evacuee_numbers))

overload_solution_matrix = solve_transportation_problem(full_cost_array, shelter_numbers, overload_evacuee_numbers)
println(overload_solution_matrix)


In [ ]:
#DERIVATIVE CALCULATION
function calculate_numerical_derivative(data::Containers.DenseAxisArray, shelter_numbers, evacuee_numbers, epsilon::Float64 = 1)
    O, D = axes(data)
    #O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        print("Calculating numerical derivative: ")
        println([i,j])
        original_cost = data[o, d]

        if original_cost isa Number
            # Increase the cost slightly and re-solve
            data[o, d] = original_cost + epsilon
            increased_cost_model = Model(HiGHS.Optimizer)
            set_silent(increased_cost_model)
            @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_cost_model, [o in O], sum(x[o, :]) == evacuee_numbers[o])
            @constraint(increased_cost_model, [d in D], sum(x[:, d]) <= shelter_numbers[d])


            optimize!(increased_cost_model)
            increased_cost_objective = objective_value(increased_cost_model)

            # Decrease the cost slightly and re-solve
            data[o, d] = original_cost - epsilon
            decreased_cost_model = Model(HiGHS.Optimizer)
            set_silent(decreased_cost_model)
            @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_cost_model, [o in O], sum(x[o, :]) == evacuee_numbers[o])
            @constraint(decreased_cost_model, [d in D], sum(x[:, d]) <= shelter_numbers[d])

            optimize!(decreased_cost_model)
            decreased_cost_objective = objective_value(decreased_cost_model)

            # Calculate the numerical derivative
            derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original cost
            data[o, d] = original_cost
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(full_cost_array, shelter_numbers, evacuee_numbers, epsilon)
println("\nNumerical Derivative Matrix for Pathway:")
for row in eachrow(derivative_matrix)
    println(row)
end

# Derivative Calculation (Shelter Capacity)

function calculate_numerical_capacity_derivative(data::Containers.DenseAxisArray,full_cost_array, shelter_numbers, evacuee_numbers, epsilon::Float64 = 1000)
    O, D = axes(data)
    #O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        print("Calculating numerical capacity derivative: ")
        println([i,j])
        original_capacity = shelter_numbers[d]
        if original_capacity isa Number
            # Increase the capacity slightly and re-solve
            shelter_numbers[d] = original_capacity + epsilon
            increased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(increased_capacity_model)
            @variable(increased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_capacity_model, [o in O], sum(x[o, :]) == evacuee_numbers[o])
            @constraint(increased_capacity_model, [d in D], sum(x[:, d]) <= shelter_numbers[d])

            optimize!(increased_capacity_model)
            increased_capacity_objective = objective_value(increased_capacity_model)

            # Decrease the capacity slightly and re-solve
            shelter_numbers[d] = original_capacity - epsilon
            decreased_capacity_model = Model(HiGHS.Optimizer)
            set_silent(decreased_capacity_model)
            @variable(decreased_capacity_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_capacity_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_capacity_model, [o in O], sum(x[o, :]) == evacuee_numbers[o])
            @constraint(decreased_capacity_model, [d in D], sum(x[:, d]) <= shelter_numbers[d])

            optimize!(decreased_capacity_model)
            decreased_capacity_objective = objective_value(decreased_capacity_model)

            # Calculate the numerical derivative
            derivative = (increased_capacity_objective - decreased_capacity_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original capacity
            shelter_numbers[d] = original_capacity
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix for shelter capacity changes
epsilon = 1000.0
capacity_derivative_matrix = calculate_numerical_capacity_derivative(full_cost_array, shelter_numbers, evacuee_numbers, epsilon)
println("\nNumerical Derivative Matrix for Shelter Capacity Changes:")
for row in eachrow(capacity_derivative_matrix)
    println(row)
end

In [120]:
full_cost_matrix = CSV.File("final_cost_matrix.csv") |> Tables.matrix

edited_cost_matrix = full_cost_matrix[:,2:end]

full_cost_array = Containers.DenseAxisArray(edited_cost_matrix, 1:990, 1:187)

num_rows, num_cols = axes(full_cost_array)
testing_counter = 0
for i in num_rows
    for j in num_cols
        if full_cost_array[i,j]>=0.50
            testing_counter+=1
        end
    end
end
print("total calculations: ")
println((size(num_rows)[1]*(size(num_cols)[1])))
print("reduced calculations: ")
println(testing_counter)
print("new total calculations: ")
println(((size(num_rows)[1]*(size(num_cols)[1]))-testing_counter))

total calculations: 185130
reduced calculations: 184808
new total calculations: 322


In [103]:
full_cost_matrix = CSV.File("final_cost_matrix.csv") |> Tables.matrix

edited_cost_matrix = full_cost_matrix[:,2:end]

maximum_distance_full_cost_array = Containers.DenseAxisArray(edited_cost_matrix, 1:991, 1:188)
num_rows, num_cols = axes(maximum_distance_full_cost_array)
num_rows = num_rows[1:end-1]
num_cols = num_cols[1:end-1]
for i in num_rows
    for j in num_cols
      try
        if maximum_distance_full_cost_array[i,j]>=0.50
           maximum_distance_full_cost_array[i,j] = "."
        end
      catch e
        println(i)
        println(j)
        println(e)
      end
    end
end
CSV.write("maximum_distance_full_cost_array.csv", Tables.table(maximum_distance_full_cost_array))

"maximum_distance_full_cost_array.csv"

In [127]:
#DONE CHECK IF EACH DISASTER LOCATION HAS A SHELTER
maximum_distance_full_cost_array_analysis = CSV.File("maximum_distance_full_cost_array.csv") |> Tables.matrix
num_rows, num_cols = axes(maximum_distance_full_cost_array)
num_rows = num_rows[1:end-1]
num_cols = num_cols[1:end-1]
for i in num_rows
  for j in num_cols
    if maximum_distance_full_cost_array_analysis[i,j] == "."
      maximum_distance_full_cost_array_analysis[i,j] = 0
      else
      maximum_distance_full_cost_array_analysis[i,j] = 1
    end
  end
end
sum_maximum_distance_full_cost_array_analysis = sum(maximum_distance_full_cost_array_analysis[1:end-1,1:end-1], dims = 2)
print("Total disaster locations with possible route: ")
print(sum(clamp.(sum_maximum_distance_full_cost_array_analysis, 0, 1)))
print("/")
print(size(num_rows)[1])

Total disaster locations with possible route: 709/990

In [61]:
full_cost_array = Containers.DenseAxisArray(edited_cost_matrix, 1:990, 1:187)
println(size(full_cost_array))
#println(num_rows)
#println(num_cols)
#println(size(edited_cost_matrix))

(991, 188)
